In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import datetime as dt
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', None)

In [3]:
# Training data sets
benef = pd.read_csv('./data/Train_Beneficiary.csv')
inpat = pd.read_csv('./data/Train_Inpatient.csv')
outpat = pd.read_csv('./data/Train_Outpatient.csv')
fraud = pd.read_csv('./data/Train-Potential Fraud.csv')

In [5]:
#Create columns for inpatient and outpatient data.
inpat['patientType'] = np.repeat('inpatient', len(inpat))
outpat['patientType'] = np.repeat('outpatient', len(outpat))

In [7]:
#Combining the inpatient and outpatient data
patient = pd.concat([inpat, outpat], axis=0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [12]:
# Combining patient dataset with beneficiary dataset 
full_data =pd.merge(patient, benef, on='BeneID', how='left')

In [21]:
full_data1 = full_data.copy()

In [22]:
# Creating DaysAdmitted Feature by substracting claim start date from claim end date 
full_data1['ClaimStartDt'] = pd.to_datetime(full_data1['ClaimStartDt'])
full_data1['ClaimEndDt'] = pd.to_datetime(full_data1['ClaimEndDt'])
full_data1['DaysAdmitted'] = full_data1['ClaimEndDt'] - full_data1['ClaimStartDt']

In [25]:
# Chaging DaysAdmitted into integer object 
full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'].astype(str)
full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'].str.replace('days', ' ')

full_data1['DaysAdmitted'] = full_data1['DaysAdmitted']. \
str.replace('00:00:00.000000000', '')

full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'].astype(int)

full_data1['DaysAdmitted'] = full_data1['DaysAdmitted'] + 1

#### First Feature Engineered: DaysAdmittedMean

In [30]:
# Creating a separate dataframe of DaysAdmittedMean by Provider
days = pd.DataFrame(full_data1.groupby('Provider')['DaysAdmitted'].mean())
days = days.reset_index()

In [35]:
# Merging Potential Fraud dataset with DaysAdmittedMean dataset 
fraud1 = fraud.copy()
fraud2 = pd.merge(fraud1, days, on='Provider')

#### 2nd Feature: DeductibleAmountPaid Mean

In [49]:
DeductibleMean = pd.DataFrame(full_data1.groupby('Provider') \
                              ['DeductibleAmtPaid'].mean())

fraud2 = pd.merge(fraud2, DeductibleMean, on='Provider')

In [93]:
fraud2.groupby('PotentialFraud')['DeductibleAmtPaid'].mean()

PotentialFraud
No     135.498511
Yes    350.839395
Name: DeductibleAmtPaid, dtype: float64

#### 3rd Feature: Insurance Claim Amount Reimbursed Mean

In [53]:
InscReimbursed = pd.DataFrame(full_data1.groupby('Provider') \
                              ['InscClaimAmtReimbursed'].mean())

fraud2 = pd.merge(fraud2, InscReimbursed, on='Provider')

In [56]:
fraud2.groupby('PotentialFraud')['InscClaimAmtReimbursed'].mean()

PotentialFraud
No     1523.780804
Yes    3842.795104
Name: InscClaimAmtReimbursed, dtype: float64

#### 4th Feature: Insurance Covered Percentage Mean

In [57]:
full_data1['InscCoveredPercent'] = full_data1['InscClaimAmtReimbursed'] \
/(full_data1['InscClaimAmtReimbursed'] + full_data1['DeductibleAmtPaid'])

InscCoveredPercent = pd.DataFrame(full_data1.groupby('Provider') \
                                  ['InscCoveredPercent'].mean())

fraud2 = pd.merge(fraud2, InscCoveredPercent, on='Provider')

In [62]:
fraud2.groupby('PotentialFraud')['InscCoveredPercent'].mean()

PotentialFraud
No     0.962077
Yes    0.934837
Name: InscCoveredPercent, dtype: float64

#### 5th Feature: Total Claim Amount Mean

In [63]:
full_data1['TotalClaimAmount'] = full_data1['InscClaimAmtReimbursed']\
+ full_data1['DeductibleAmtPaid']

TotalClaimAmount = pd.DataFrame(full_data1.groupby('Provider') \
                                ['TotalClaimAmount'].mean())

fraud2 = pd.merge(fraud2, TotalClaimAmount, on='Provider')

In [66]:
fraud2.groupby('PotentialFraud')['TotalClaimAmount'].mean()

PotentialFraud
No     1645.778736
Yes    4142.438122
Name: TotalClaimAmount, dtype: float64

#### 6th Feature: Daily Total Charge

In [69]:
full_data1['DailyTotalCharge'] = full_data1['TotalClaimAmount'] \
/full_data1['DaysAdmitted']

DailyTotalCharge = pd.DataFrame(full_data1.groupby('Provider') \
                                ['DailyTotalCharge'].mean())

fraud2 = pd.merge(fraud2, DailyTotalCharge, on='Provider')

In [72]:
fraud2.groupby('PotentialFraud')['DailyTotalCharge'].mean()

PotentialFraud
No     471.273799
Yes    952.785225
Name: DailyTotalCharge, dtype: float64

#### Features 7-10: Annual Reimbursement and Deductible Paid Amounts for Inpatient and Outpatient

In [75]:
IPAnnReimbMean = pd.DataFrame(full_data1.groupby('Provider') \
                              ['IPAnnualReimbursementAmt'].mean())

IPAnnDeductMean = pd.DataFrame(full_data1.groupby('Provider') \
                               ['IPAnnualDeductibleAmt'].mean())

OPAnnReimbMean = pd.DataFrame(full_data1.groupby('Provider') \
                              ['OPAnnualReimbursementAmt'].mean())

OPAnnDeductMean = pd.DataFrame(full_data1.groupby('Provider') \
                               ['OPAnnualDeductibleAmt'].mean())

In [77]:
fraud2 = pd.merge(fraud2, IPAnnReimbMean, on='Provider')
fraud2 = pd.merge(fraud2, IPAnnDeductMean, on='Provider')
fraud2 = pd.merge(fraud2, OPAnnReimbMean, on='Provider')
fraud2 = pd.merge(fraud2, OPAnnDeductMean, on='Provider')

In [79]:
fraud2.groupby('PotentialFraud')['IPAnnualReimbursementAmt'].mean()

PotentialFraud
No     5868.975601
Yes    9052.076164
Name: IPAnnualReimbursementAmt, dtype: float64

In [80]:
fraud2.groupby('PotentialFraud')['IPAnnualDeductibleAmt'].mean()

PotentialFraud
No     638.107001
Yes    946.817676
Name: IPAnnualDeductibleAmt, dtype: float64

In [81]:
fraud2.groupby('PotentialFraud')['OPAnnualReimbursementAmt'].mean()

PotentialFraud
No     2191.723203
Yes    2273.113210
Name: OPAnnualReimbursementAmt, dtype: float64

In [82]:
fraud2.groupby('PotentialFraud')['OPAnnualDeductibleAmt'].mean()

PotentialFraud
No     628.840769
Yes    631.565539
Name: OPAnnualDeductibleAmt, dtype: float64

#### Feature 11: Total Annual Claim Amount Mean

In [84]:
full_data1['TotalAnnClaimAmt'] = full_data1['IPAnnualReimbursementAmt']+ \
full_data1['IPAnnualDeductibleAmt']+full_data1['OPAnnualReimbursementAmt'] \
+ full_data1['OPAnnualDeductibleAmt']

TotalAnnClaimAmt = pd.DataFrame(full_data1.groupby('Provider') \
                                ['TotalAnnClaimAmt'].mean())

fraud2 = pd.merge(fraud2, TotalAnnClaimAmt, on='Provider')

In [87]:
fraud2.groupby('PotentialFraud')['TotalAnnClaimAmt'].mean()

PotentialFraud
No      9327.646575
Yes    12903.572589
Name: TotalAnnClaimAmt, dtype: float64

In [88]:
fraud2

,Provider,PotentialFraud,DaysAdmitted,DeductibleAmtPaid,InscClaimAmtReimbursed,InscCoveredPercent,TotalClaimAmount,DailyTotalCharge,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,TotalAnnClaimAmt
0,PRV51001,No,2.440000,213.600000,4185.600000,0.975656,4399.200000,1061.072000,17606.000000,897.120000,2615.200000,463.920000,21582.240000
1,PRV51003,Yes,4.674242,502.166667,4588.409091,0.912167,5090.575758,1298.615410,7568.181818,931.424242,2678.181818,737.121212,11914.909091
2,PRV51004,No,2.429530,2.080537,350.134228,0.978485,352.214765,250.363050,4351.879195,434.953020,2194.899329,622.751678,7604.483221
3,PRV51005,Yes,2.088412,3.175966,241.124464,0.980747,244.300429,196.533055,3623.991416,379.162232,2109.733906,636.328755,6749.216309
4,PRV51007,No,1.958333,45.333333,468.194444,0.985303,513.527778,243.681944,3050.000000,445.000000,1729.722222,469.722222,5694.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,3.142857,4.642857,380.000000,0.983401,384.642857,200.629252,3962.142857,457.714286,3241.785714,886.785714,8548.428571
5406,PRV57760,No,1.318182,0.000000,216.818182,1.000000,216.818182,216.022727,2785.454545,436.909091,1492.727273,805.454545,5520.545455
5407,PRV57761,No,2.390244,4.512195,225.243902,0.935979,229.756098,157.134674,7026.585366,586.097561,2928.414634,707.317073,11248.414634
5408,PRV57762,No,1.000000,0.000000,1900.000000,1.000000,1900.000000,1900.000000,15000.000000,1068.000000,2540.000000,400.000000,19008.000000


## Randy's Features

#### Add number of doctors

In [94]:
doctors= full_data1.groupby(['Provider', 'AttendingPhysician']) \
['AttendingPhysician'].count().reset_index(name='NumOfDoctors'). \
groupby('Provider')['NumOfDoctors'].count().reset_index()

fraud2 = pd.merge(fraud2, doctors, on='Provider')

#### Add number of patients

In [97]:
patient= full_data.groupby(['Provider','BeneID'])['BeneID'].count(). \
reset_index(name='NumOfPatients').groupby('Provider')['NumOfPatients']. \
count().reset_index()

fraud2 = pd.merge(fraud2, patient, on='Provider')

#### Add Type of Service

In [101]:
a = full_data1.groupby(['Provider', 'patientType'])['BeneID'].count().reset_index(name='count').\
drop('count', axis=1)

a_list= list(zip(a.Provider, a.patientType))

providerDict= {}

for ele in a_list:
    if ele[0] not in providerDict:
        providerDict[ele[0]]= ele[1]    
    else:
        providerDict[ele[0]] = 'both'
        

providerService = pd.DataFrame(providerDict.keys(), providerDict.values()).reset_index().\
rename(columns={'index':'ServiceType', 0:'Provider'})

In [102]:
fraud2 = pd.merge(fraud2, providerService, on='Provider')

#### Add number of claims

In [103]:
claims= full_data1.groupby(['Provider', 'ClaimID'])['ClaimID'].count().\
reset_index(name='NumOfClms').groupby('Provider')['NumOfClms'].count().reset_index()

fraud2 = pd.merge(fraud2, claims, on='Provider')

#### Add weekly claims 

In [105]:
full_data1['startClaimWeek']= full_data1['ClaimStartDt'].dt.week

In [106]:
weeklyClaims= full_data1.groupby(['Provider','startClaimWeek', 'ClaimID'])['ClaimID'].count().reset_index(name='WeeklyClaims').\
groupby(['Provider', 'startClaimWeek'])['WeeklyClaims'].count().reset_index()

In [107]:
fraud3 = fraud2.copy()

In [108]:
fraud3= pd.merge(fraud3, weeklyClaims, on='Provider', how='left')

#### Add average duration of treatment per claims

#### Add monthly claims

In [118]:
full_data1['startClaimMonth'] = full_data1['ClaimStartDt'].dt.month

monthly = full_data1.groupby(['Provider','startClaimMonth', 'ClaimID'])['ClaimID'].count().reset_index(name='MonthlyClaims').\
groupby(['Provider', 'startClaimMonth'])['MonthlyClaims'].count().reset_index()

In [119]:
fraud3 = pd.merge(fraud3, monthly, on='Provider', how='left')

In [121]:
fraud3.to_pickle('newfeatures_df.pkl')